# PeDistance Linear Regression

### Initial settings

In [75]:
import numpy as np
import torch
from torch import nn
import pandas as pd
from torch.utils.tensorboard import SummaryWriter
    
def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu') # don't have GPU 
    return device

# convert a df to tensor to be used in pytorch
def df_to_tensor(df):
    device = get_device()
    return torch.tensor(df.to_numpy()).float().to(device)

### Import Dataset

In [2]:
fields = [
    'Distance', 
    'PedTopLeftX', 
    'PedTopLeftY', 
    'PedTopRightX', 
    'PedTopRightY',
    'PedBottomLeftX', 
    'PedBottomLeftY',
    'PedBottomRightX',
    'PedBottomRightY'
]

df = pd.read_csv (r'./Dataset.csv', usecols=fields)

In [63]:
print(df.shape)

(150, 9)


### Parsing raw dataset data

In [65]:
Y_target = df.Distance
print(Y_target.shape)

(150,)


In [70]:
X_data = df.apply(lambda el :  pd.Series([abs(el.PedTopRightX - el.PedTopLeftX), abs(el.PedBottomLeftY - el.PedTopLeftY)], index=['width', 'height']), axis=1)
print(X_data)

     width  height
0     62.0   187.0
1     65.0   138.0
2    159.0   420.0
3     97.0   203.0
4     49.0   121.0
..     ...     ...
145  155.0   395.0
146  157.0   380.0
147   50.0   150.0
148   55.0   131.0
149   52.0   129.0

[150 rows x 2 columns]


### Splitting dataset

In [72]:
X_training = df_to_tensor(X_data[:75])
Y_training = df_to_tensor(Y_target[:75])

X_testing = df_to_tensor(X_data[75:])
Y_testing =  df_to_tensor(Y_target[75:])
print(X_training.shape)
print(Y_training.shape)
print()
print(X_testing.shape)
print(Y_testing.shape)

torch.Size([75, 2])
torch.Size([75])

torch.Size([75, 2])
torch.Size([75])


In [73]:
class LinearRegressor(nn.Module):
    def __init__(self, in_size, out_size):
        super(LinearRegressor, self).__init__()
        self.linear = nn.Linear(in_size,out_size)
    
    def forward(self,x):
        result = self.linear(x)
        return result

In [85]:
lr = 0.0001

epochs = 200000

writer = SummaryWriter('logs/linear_regressor_lr' + str(lr) + '_epochs' + str(epochs))

#normalizzazione dei dati
means = X_training.mean(0)
stds = X_training.std(0)

X_training_norm = (X_training-means)/stds
X_testing_norm = (X_testing-means)/stds

reg = LinearRegressor(2,1).cuda()
criterion = nn.MSELoss()

optimizer = torch.optim.SGD(reg.parameters(),lr=lr)

for e in range(epochs):
    reg.train()
    output = reg(X_training_norm)
    l = criterion(output.view(-1),Y_training)

    writer.add_scalar('loss/train', l.item(), global_step=e)
    
    l.backward()
    
    optimizer.step()    
    optimizer.zero_grad()
    
    reg.eval()
 
    with torch.set_grad_enabled(False):
        y_test = reg(X_testing_norm)
        l = criterion(y_test.view(-1),Y_testing)
        writer.add_scalar('loss/test', l.item(), global_step=e)